# Imports and Function definitions

In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 41.9 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ub

In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(text, id, rmv_stopwords=True):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  result = []
  if rmv_stopwords:
    counter = Counter([i for i in tokens if i not in all_stopwords])
  else:
    counter = Counter([i for i in tokens])
    
  for key,val in counter.items():
    tup_to_append = (key, (id, val))
    result.append(tup_to_append)
    
  return result

In [5]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl)
  

  

In [6]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  result = postings.map(lambda x : (x[0], len(x[1])))
  return result


In [7]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  result_before_grouping = postings.map(lambda x : (token2bucket_id(x[0]), x))
  result_after_grouping = result_before_grouping.groupByKey()
  return result_after_grouping.map(InvertedIndex.write_a_posting_list)
  
  


In [8]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

# Inverted Index creation - body

In [9]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [10]:
#Choose the correct file to load - from the google bucket or from the csv file

#Google bucket
# doc_title_pairs = parquetFile.select("title", "id").rdd
# from pathlib import Path 
# import os

# try:
#     if os.environ["wikidata_preprocessed"] is not None:
#       path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
# except:
#       path = "wikidumps/*"

#CSV File
parquetFile_pd = pd.read_csv("small_corpus.csv")
parquetFile = spark.createDataFrame(parquetFile_pd)
parquetFile.show()

# parquetFile = spark.read.parquet(path)
# print(type(parquetFile))

+----------+-------+--------------------+--------------------+--------------------+
|Unnamed: 0|     id|               title|                text|         anchor_text|
+----------+-------+--------------------+--------------------+--------------------+
|         0|4055486|         Time Walker|'''''Time Walker'...|[Row(id=34240044,...|
|         1|4080953|Nazi racial theories|The Nazi Party ad...|[Row(id=21736, te...|
|         2|4093674|              Siopao|'''''Siopao''''' ...|[Row(id=232026, t...|
|         3|4105482|       Plain vanilla|'''Plain vanilla'...|[Row(id=37512, te...|
|         4|4108839|          NBA on TNT|'''''NBA on TNT''...|[Row(id=22093, te...|
|         5|4141563|Predictive analytics|'''Predictive ana...|[Row(id=26685, te...|
|         6|4146044|        Shrimp toast|'''Shrimp toast''...|[Row(id=13404, te...|
|         7|4148655|       Jeremy Renner|'''Jeremy Lee Ren...|[Row(id=843158, t...|
|         8|4169317|          Goat tying|thumb|Goat tying ...|[Row(id=130105

In [34]:
doc_anchor_text_pairs = parquetFile.select("anchor_text", "id")


dict_of_anchor_words = {}

# Small corpus represents the anchor text as a string - so we extract from it the anchor and the id
# Return dict of {word: (doc_id, 1), (doc_id, 1)}
for row in doc_anchor_text_pairs.collect(): 
  
  anchor_as_list = re.sub("(\[Row\(id=)|(Row\(id=)|(, text=')|('\), )|('\)])", ",",row.anchor_text).split(",")
  id = None
  for i in anchor_as_list:
    
    if i is "":
      continue

    elif i.isnumeric():
      id = i
 
    else:
      for word in i.split(' '):
        word = word.lower()
        if word in all_stopwords:
          continue
        if word not in dict_of_anchor_words.keys():
          dict_of_anchor_words[word] = Counter()
          dict_of_anchor_words[word][id] = 1
        else:
          dict_of_anchor_words[word][id] += 1


# Code for normal corpus - will hold dict of {word: (doc_id, 1), (doc_id, 1)}
# for row in doc_text_pairs.collect():
#   for pair in row.anchor_text:
#     for word in pair.text.split(' '):
#       word = word.lower()
      
#       if word in all_stopwords:
#         continue

#       if word not in dict_of_anchor_words.keys():
#         dict_of_anchor_words[word] = Counter()
#         dict_of_anchor_words[word][pair.id] = 1
      
#       else:
#         dict_of_anchor_words[word][pair.id] += 1

posting_anchor_text = list(map(lambda x: (x[0], [(int(i[0]), i[1]) for i in x[1].most_common()]), list(dict_of_anchor_words.items())))
print(posting_anchor_text[0])




('tom', [(26763420, 27), (531828, 24), (15003874, 20), (719414, 17), (31460, 15), (43568, 15), (367423, 9), (17130725, 8), (2537370, 7), (277451, 7), (314665, 6), (256830, 5), (2184365, 5), (30265, 4), (2874053, 4), (2315112, 4), (715939, 4), (1568960, 4), (213394, 3), (8861986, 3), (47770304, 3), (6632427, 3), (404895, 3), (1352088, 3), (339841, 3), (216169, 3), (448514, 3), (2066558, 3), (350713, 3), (2702913, 3), (17490545, 3), (9351661, 3), (19454441, 3), (34240044, 2), (3491110, 2), (56796335, 2), (623013, 2), (50539648, 2), (727542, 2), (1086815, 2), (1792975, 2), (12926429, 2), (295706, 2), (2199999, 2), (4113657, 2), (99628, 2), (44203434, 2), (253555, 2), (663818, 2), (414308, 2), (148556, 2), (4712384, 2), (33938394, 2), (5861398, 1), (57285152, 1), (43135807, 1), (26098493, 1), (953881, 1), (12764766, 1), (36049969, 1), (30387, 1), (2343249, 1), (6304971, 1), (168507, 1), (97732, 1), (66846904, 1), (103326, 1), (52207008, 1), (1613833, 1), (2208661, 1), (32895937, 1), (15565

In [35]:
posting_anchor_text = spark.sparkContext.parallelize(posting_anchor_text)
print(type(posting_anchor_text))
print(posting_anchor_text.first())
anchor_text_df = calculate_df(posting_anchor_text)
w2df_dict_anchor_text = anchor_text_df.collectAsMap()

<class 'pyspark.rdd.RDD'>
('tom', [(26763420, 27), (531828, 24), (15003874, 20), (719414, 17), (31460, 15), (43568, 15), (367423, 9), (17130725, 8), (2537370, 7), (277451, 7), (314665, 6), (256830, 5), (2184365, 5), (30265, 4), (2874053, 4), (2315112, 4), (715939, 4), (1568960, 4), (213394, 3), (8861986, 3), (47770304, 3), (6632427, 3), (404895, 3), (1352088, 3), (339841, 3), (216169, 3), (448514, 3), (2066558, 3), (350713, 3), (2702913, 3), (17490545, 3), (9351661, 3), (19454441, 3), (34240044, 2), (3491110, 2), (56796335, 2), (623013, 2), (50539648, 2), (727542, 2), (1086815, 2), (1792975, 2), (12926429, 2), (295706, 2), (2199999, 2), (4113657, 2), (99628, 2), (44203434, 2), (253555, 2), (663818, 2), (414308, 2), (148556, 2), (4712384, 2), (33938394, 2), (5861398, 1), (57285152, 1), (43135807, 1), (26098493, 1), (953881, 1), (12764766, 1), (36049969, 1), (30387, 1), (2343249, 1), (6304971, 1), (168507, 1), (97732, 1), (66846904, 1), (103326, 1), (52207008, 1), (1613833, 1), (2208661,

In [36]:
posting_locs_list_anchor_text = partition_postings_and_write(posting_anchor_text).collect()

In [37]:
# merge the posting locations into a single dict
super_posting_locs_anchor_text = defaultdict(list)
for posting_loc in posting_locs_list_anchor_text:
  for k, v in posting_loc.items():
    super_posting_locs_anchor_text[k].extend(v)

In [38]:
# # Create inverted index instance for anchor
inverted_anchor_text = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_anchor_text.posting_locs = super_posting_locs_anchor_text
# Add the token - df dictionary to the inverted index
inverted_anchor_text.df = w2df_dict_anchor_text
# write the global stats out
inverted_anchor_text.write_index('.', 'index_anchor')

#Test

In [ ]:
## test the final index
# pl = read_posting_list(inverted_text, 'clarity')
# print(pl)

pl1 = read_posting_list(inverted_anchor_text, 'clinical psychology')
print(pl1)

KeyError: ignored